<a href="https://colab.research.google.com/github/marcinmichalsmolen/ZUM/blob/main/INL_MARCIN_SMOLEN_Poleval_2019_Task6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INL PROJEKT Marcin Smoleń s23429 Poleval 2019 Task 6.1 i 6.2 - Automatic cyberbullying detection

W ciagu ostatniej dekady znaczący wzrost popularności serwisów
społecznościowych pokazał jak poważnym problemem jest cyberprzemoc.
Zbiór danych służący do trenowania i oceny prezentowanych systemów
zawiera tweety zebrane z ogólnodostepnych dyskusji na Twitterze z
zastosowanyniem anonimizacji postów. W tym zadaniu pilotażowym
zawodnicy określa, czy wpis internetowy można zakwalifikować jako część
narracji o cyberprzemocy, czy nie. Wpisy beda zawiera ly tweety zebrane z
ogólnodostepnych dyskusji na Twitterze. Ponieważ duża część problemu
automatycznego wykrywania cyberprzemocy czesto opiera sie na wyborze
metody przetwarzania, tweety zostały dostarczone jako takie, z
minimalnym przetwarzaniem wstepnym. Cyberprzemoc to działania
skierowane do osób prywatnych. Mowa nienawieści skierowana jest do
osoby publicznej, podmioty lub dużej grupy ludzi. W obydwóch zadaniach do obliczeń zostały wykorzystane te same tweety. W pierwszym zadaniu podzielone są na 2 kategorie a w drugim na 3.

**POLEVAL 2019 wzor Task 6: Automatic cyberbullying detection http://2019.poleval.pl/index.php/tasks/task6 - Subtask 6.1**


Task 6.1 - 10041 tweetów treningowych w 2 kategoriach:
nieszkodliwe i szkodliwe, dane testowe 1000 tweetów ze zbiorem tagów
do weryfikacji

Import potrzebnych biliotek


In [16]:
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import ipykernel
'exec(%matplotlib inline)'
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
nltk.download("popular")


from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import (accuracy_score, confusion_matrix, f1_score, precision_score, recall_score)
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB, ComplementNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
import sklearn.naive_bayes

!pip install stop_words
from stop_words import get_stop_words
stopWords = get_stop_words('pl')

!pip install pystempel
from stempel import StempelStemmer
stemmer = StempelStemmer.polimorf()

import string
from sklearn.utils import shuffle



[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Loading: 100%|██████████| 11368252/11368252 [00:12<00:00, 911659.29bytes/s]


Zdefinijuje funkcje potrzebne do wyczyszczenia setu danych.

W secie mamy ponad 10km tweetow. Wszystkie zaczynaja sie nazwa uzytkownika ktora zostala zanonimizona wiec mozna ja spokojnie usunac.

Usuwamy stopwords, czyli slowa ktore nie maja zadnego znaczenia i nie wnoszą nic do tekstu z punktu widzenia logiki i przekazu. np. a, aby, nigdy, wtedy, wy.

Usuwamy znaki specjalne, czyli !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~

gdy juz mamy wyczyszczone dane przeprowadzamy stemming, czyli wydobywamy ze słowa jego podstawę, np. jechac, pojechac, jechalem, pojachalbym
 =zamienia sie na => "jecha"

In [17]:
def removeUserNames(text):
    text['text'] = text['text'].str.replace('@anonymized_account', '')
    return text

def removeStopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in stopWords]
    return " ".join(text)

def removeSpecialChars(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

def stem(text):
    text = [stemmer.stem(word) for word in text.split()]
    a = [i for i in text if i]
    return " ".join(a)

In [18]:
y_train = pd.read_csv('http://77.55.209.58/inl/task1/training_set_clean_only_tags.txt', delimiter='\t', names=['y'])
X_train = pd.read_csv('http://77.55.209.58/inl/task1/training_set_clean_only_text.txt', delimiter='\t', names=['text'])

In [19]:
X_train

,text
0,Dla mnie faworytem do tytułu będzie Cracovia. ...
1,@anonymized_account @anonymized_account Brawo ...
2,"@anonymized_account @anonymized_account Super,..."
3,@anonymized_account @anonymized_account Musi. ...
4,"Odrzut natychmiastowy, kwaśna mina, mam problem"
...,...
10036,@anonymized_account Ty zagrasz? Nie wiedziałem 😉
10037,@anonymized_account @anonymized_account A VAR ...
10038,@anonymized_account @anonymized_account Szanow...
10039,@anonymized_account @anonymized_account @anony...


In [20]:
len(y_train)

10041

In [21]:
type(X_train)

pandas.core.frame.DataFrame

In [22]:
X_train = removeUserNames(X_train)
X_train['text'] = X_train['text'].apply(removeSpecialChars).apply(removeStopwords).apply(stem)
X_train = X_train['text'].values.tolist()

In [23]:
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)

In [24]:
X_train[1]

<1x14382 sparse matrix of type '<class 'numpy.int64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [25]:
transformer = TfidfTransformer()
X_train = transformer.fit_transform(X_train)

In [26]:
X_train[1]

<1x14382 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements in Compressed Sparse Row format>

**MultinomialNB**

Algorytm wielomianowego naiwnego Bayesa to podejście bayesowskie do uczenia się popularne w przetwarzaniu języka naturalnego (NLP). Program odgaduje znacznik tekstu, takiego jak e-mail lub artykuł w gazecie, korzystając z twierdzenia Bayesa. Oblicza prawdopodobieństwo każdego znacznika dla danej próbki i wyprowadza znacznik z największą szansą.


In [27]:
model = MultinomialNB()
model.fit(X_train, y_train)

y_predict_train = model.predict(X_train)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [28]:
print('Acc: ', accuracy_score(y_train,  y_predict_train))

Acc:  0.9168409520964047


In [29]:
print('F1:', f1_score(y_train,  y_predict_train, average = 'weighted'))

F1: 0.8786029464134026


In [30]:
y_test = pd.read_csv('http://77.55.209.58/inl/test1/test_set_clean_only_tags.txt', delimiter='\t', names=['y'])
X_test = pd.read_csv('http://77.55.209.58/inl/test1/test_set_clean_only_text.txt', delimiter='\t', names=['text'])

In [31]:
len(y_test)

1000

In [32]:
len(X_test)

1000

In [33]:
X_test = removeUserNames(X_test)
X_test['text'] = X_test['text'].apply(removeSpecialChars).apply(removeStopwords).apply(stem)
X_test = X_test['text'].values.tolist()

In [34]:
X_test[1]

'ale mieć szansa zagranie ativus ten proba czysty prowizorka'

In [35]:
X_test = vectorizer.transform(X_test)
X_test[1]

<1x14382 sparse matrix of type '<class 'numpy.int64'>'
	with 7 stored elements in Compressed Sparse Row format>

In [36]:
X_test = transformer.transform(X_test)
X_test[1]

<1x14382 sparse matrix of type '<class 'numpy.float64'>'
	with 7 stored elements in Compressed Sparse Row format>

In [37]:
y_predict_test = model.predict(X_test)

In [38]:
print('Acc:', accuracy_score(y_test,  y_predict_test))

Acc: 0.866


In [39]:
print('F1:', f1_score(y_test,  y_predict_test, average = 'weighted'))

F1: 0.8038113612004287


**BernoulliNB**
 BernoulliNB implementuje naiwne algorytmy uczenia i klasyfikacji Bayesa dla danych, które są dystrybuowane zgodnie z wielowymiarowymi rozkładami Bernoulliego; tzn. może istnieć wiele cech, ale zakłada się, że każda z nich jest zmienną o wartości binarnej (Bernoulli, boolean).




In [40]:
model = BernoulliNB()
model.fit(X_train, y_train)

y_predict_train = model.predict(X_train)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [41]:
print('Acc: ', accuracy_score(y_train,  y_predict_train))

Acc:  0.9242107359824718


In [42]:
print('F1:', f1_score(y_train,  y_predict_train, average = 'weighted'))

F1: 0.8954942598431733


In [43]:
y_predict_test = model.predict(X_test)

In [44]:
print('Acc:', accuracy_score(y_test,  y_predict_test))

Acc: 0.866


In [45]:
print('F1:', f1_score(y_test,  y_predict_test, average = 'weighted'))

F1: 0.8038113612004287



**POLEVAL 2019 wzor Task 6: Automatic cyberbullying detection http://2019.poleval.pl/index.php/tasks/task6 - Subtask 6.2**

Task 6.2 - 10041 tweet´ow treningowych w 3 kategoriach:
nieszkodliwe ,cyberprzemoc i mowa nienawiści, dane testowe 1000
tweet´ow z zbiorów tagów do weryfikacji

W tym zadaniu uzyje **XGBoostClassifier**

XGBoost to implementacja algorytmu drzewa gradientowego, który jest powszechnie uznawany ze wzgledu wydajność i dokładność predykcyjną. 

In [46]:
!pip install xgboost
from xgboost import XGBClassifier

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [49]:
y_train = pd.read_csv('http://77.55.209.58/inl/task2/training_set_clean_only_tags.txt', delimiter='\t', names=['y'])
X_train = pd.read_csv('http://77.55.209.58/inl/task2/training_set_clean_only_text.txt', delimiter='\t', names=['text'])
y_test = pd.read_csv('http://77.55.209.58/inl/test2/test_set_only_tags.txt', delimiter='\t', names=['y'])
X_test = pd.read_csv('http://77.55.209.58/inl/test2/test_set_only_text.txt', delimiter='\t', names=['text'])

In [50]:
X_train = removeUserNames(X_train)
X_train['text'] = X_train['text'].apply(removeSpecialChars).apply(removeStopwords).apply(stem)
X_train = X_train['text'].values.tolist()

In [51]:
vectorizer = CountVectorizer()
transformer = TfidfTransformer()

X_train = vectorizer.fit_transform(X_train)
X_train = transformer.fit_transform(X_train)

In [52]:
model = XGBClassifier()
model.fit(X_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(objective='multi:softprob')

In [53]:
y_predict_train = model.predict(X_train)
print('Acc:', accuracy_score(y_train,  y_predict_train))

Acc: 0.9180360521860372


In [54]:
print('F1:', f1_score(y_train,  y_predict_train, average='weighted'))

F1: 0.8816011287626483


In [55]:
X_test = removeUserNames(X_test)
X_test['text'] = X_test['text'].apply(removeSpecialChars).apply(removeStopwords).apply(stem)
X_test = X_test['text'].values.tolist()

In [56]:
X_test = vectorizer.transform(X_test)
X_test = transformer.transform(X_test)

In [57]:
y_predict_test = model.predict(X_test)
print('Acc:', accuracy_score(y_test,  y_predict_test))

Acc: 0.868


In [58]:
print('F1:', f1_score(y_test, y_predict_test, average='weighted'))

F1: 0.8086017476704171
